<a href="https://colab.research.google.com/github/LucGtff/explainability/blob/main/gamma_facet_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install bcg-facet


ERROR: Could not find a version that satisfies the requirement bcg-facet (from versions: none)
ERROR: No matching distribution found for bcg-facet


In [3]:
!pip install gamma-facet


     |████████████████████████████████| 71kB 3.4MB/s 
     |████████████████████████████████| 51kB 3.8MB/s 
     |████████████████████████████████| 1.8MB 5.8MB/s 
     |████████████████████████████████| 276kB 18.7MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 81kB 6.5MB/s 
     |████████████████████████████████| 645kB 19.1MB/s 
     |████████████████████████████████| 307kB 25.8MB/s 
  Created wheel for shap: filename=shap-0.35.0-cp37-cp37m-linux_x86_64.whl size=395328 sha256=d77d60d7b5cc241a13544bc60d8259afcb0ae6e1e4944a1b5e05de125c7bcb63
  Stored in directory: /root/.cache/pip/wheels/e7/f7/0f/b57055080cf8894906b3bd3616d2fc2bfd0b12d5161bcb24ac
Successfully built shap
  Found existing installation: joblib 1.0.1
    Uninstalling joblib-1.0.1:
      Successfully uninstalled joblib-1.0.1
  Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3
  Found existing installati

In [24]:
# standard imports
import pandas as pd
from sklearn.model_selection import RepeatedKFold


# some helpful imports from sklearndf
from sklearndf.pipeline import RegressorPipelineDF
from sklearndf.regression import RandomForestRegressorDF

# relevant FACET imports
from facet.data import Sample
from facet.selection import LearnerRanker, LearnerGrid



from google.colab import files 
  
  
uploaded = files.upload()


Saving diabetes_quickstart_1.csv to diabetes_quickstart_1.csv


In [25]:
import pandas as pd 
import io 
  
diabetes_df = pd.read_csv(io.BytesIO(uploaded['diabetes_quickstart_1.csv'])) 
print(diabetes_df)

     AGE  SEX   BMI      BP   S1     S2    S3    S4      S5   S6    Y
0     59    2  32.1  101.00  157   93.2  38.0  4.00  4.8598   87  151
1     48    1  21.6   87.00  183  103.2  70.0  3.00  3.8918   69   75
2     72    2  30.5   93.00  156   93.6  41.0  4.00  4.6728   85  141
3     24    1  25.3   84.00  198  131.4  40.0  5.00  4.8903   89  206
4     50    1  23.0  101.00  192  125.4  52.0  4.00  4.2905   80  135
..   ...  ...   ...     ...  ...    ...   ...   ...     ...  ...  ...
437   60    2  28.2  112.00  185  113.8  42.0  4.00  4.9836   93  178
438   47    2  24.9   75.00  225  166.0  42.0  5.00  4.4427  102  104
439   60    2  24.9   99.67  162  106.6  43.0  3.77  4.1271   95  132
440   36    1  30.0   95.00  201  125.2  42.0  4.79  5.1299   85  220
441   36    1  19.6   71.00  250  133.2  97.0  3.00  4.5951   92   57

[442 rows x 11 columns]


In [27]:
# create FACET sample object
diabetes_sample = Sample(observations=diabetes_df, target_name="Y")

print(diabetes_sample)

In [26]:
# create a (trivial) pipeline for a random forest regressor
rnd_forest_reg = RegressorPipelineDF(
    regressor=RandomForestRegressorDF(random_state=42)
)

# define grid of models which are "competing" against each other
rnd_forest_grid = [
    LearnerGrid(
        pipeline=rnd_forest_reg,
        learner_parameters={
            "min_samples_leaf": [8, 11, 15]
        }
    ),
]

# create repeated k-fold CV iterator
rkf_cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)

# rank your candidate models by performance (default is mean CV score - 2*SD)
ranker = LearnerRanker(
    grids=rnd_forest_grid, cv=rkf_cv, n_jobs=-3
).fit(sample=diabetes_sample)

# get summary report
ranker.summary_report()

ValueError: ignored